In [ ]:
import time
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter


from helper_funcs import read_txt_file_to_string

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10)

doc1 = read_txt_file_to_string("documents/doc1.txt")[:10]
doc2 = read_txt_file_to_string("documents/doc2.txt")[:10]
print(doc1)
documents = [
    Document(page_content=doc1, metadata={"source": "doc1.txt"}),
    Document(page_content=doc2, metadata={"source": "doc2.txt"}),
]
splits = splitter.split_documents(documents)

# --- Initialize or load Chroma DB --- https://python.langchain.com/docs/integrations/vectorstores/chroma/
vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",
)

def safe_add_documents(vector_store, docs, delay=10):
    print(docs)
    vector_store.add_documents(docs)
    time.sleep(delay)


safe_add_documents(vector_store, splits)


[Document(metadata={'source': 'doc1.txt'}, page_content='Document A'), Document(metadata={'source': 'doc2.txt'}, page_content='Document B')]


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers.string import StrOutputParser
import json

prompt_template = ChatPromptTemplate.from_template("""
Je bent een compliance-expert gespecialiseerd in DORA, ISO27001 en vergelijkbare standaarden.
Vergelijk de twee onderstaande documenten per criterium.
Geef per criterium:
- een samenvatting van hoe elk document het onderwerp behandelt
- een beoordeling van de mate van overeenstemming (bijv. "Volledig in lijn", "Gedeeltelijk", "Niet in lijn")
- een korte toelichting
- citeer indien mogelijk relevante fragmenten uit beide documenten.

Document A:
\"\"\"{doc1}\"\"\"

Document B:
\"\"\"{doc2}\"\"\"

Criteria:
{criteria}

Antwoord in geldig JSON-formaat als volgt:
[
    {{
    "criterium": "...",
    "overeenstemming": "...",
    "analyse_document_A": "...",
    "analyse_document_B": "...",
    "toelichting": "...",
    "referentie_A": "...",
    "referentie_B": "..."
    }},
    ...
]
""")

# 5️⃣ Test data
doc1 = "hi"
doc2 = "hoi"
criteria = "Governance structuur\nRisicomanagement\nIncident response\nOutsourcing beleid"

# 6️⃣ Run the chain
chain = prompt_template | llm | StrOutputParser()

output = chain.invoke({
    "doc1": doc1,
    "doc2": doc2,
    "criteria": json.dumps(criteria, ensure_ascii=False)
})

print(output)

```json
[
    {
        "criterium": "Governance structuur",
        "overeenstemming": "Niet in lijn",
        "analyse_document_A": "Document A biedt geen informatie over de governance structuur.",
        "analyse_document_B": "Document B bevat ook geen details over de governance structuur.",
        "toelichting": "Beide documenten missen essentiële informatie over hoe de governance structuur is opgezet, wat cruciaal is voor compliance met DORA en ISO27001.",
        "referentie_A": "N.v.t.",
        "referentie_B": "N.v.t."
    },
    {
        "criterium": "Risicomanagement",
        "overeenstemming": "Niet in lijn",
        "analyse_document_A": "Document A behandelt risicomanagement niet.",
        "analyse_document_B": "Document B biedt ook geen richtlijnen of informatie over risicomanagement.",
        "toelichting": "Zonder een duidelijk risicomanagementbeleid voldoen beide documenten niet aan de vereisten van DORA en ISO27001.",
        "referentie_A": "N.v.t.",
        "r